# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
load_csv = "../WeatherPy/output_data/cities.csv"
cities = pd.read_csv(load_csv)
cities.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,0,padang,83,ID,1608481781,68,-0.95,100.35,78.15,3.74
1,1,mount isa,75,AU,1608481781,100,-20.73,139.50,75.20,4.70
2,2,jamestown,90,US,1608481641,98,42.10,-79.24,35.01,9.17
3,3,faanui,75,PF,1608481781,70,-16.48,-151.75,80.17,5.61
4,4,demyansk,99,RU,1608481782,94,57.64,32.47,35.06,6.46


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities[['Lat', 'Lng']]

# Store Humidity in humidity
humidity = cities['Humidity']

In [8]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Create new DataFrame fitting weather criteria
# max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

focus_cities = cities.copy()
focus_cities = focus_cities[focus_cities['Max Temperature'] >= 70]
focus_cities = focus_cities[focus_cities['Max Temperature'] <= 80]  
focus_cities = focus_cities[focus_cities['Wind Speed'] <= 10]
focus_cities = focus_cities[focus_cities['Cloudiness'] == 0]
focus_cities = focus_cities.dropna()
focus_cities = focus_cities.reset_index()
focus_cities

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,65,65,vaini,0,TO,1608481791,94,-21.20,-175.20,77.00,8.05
1,100,100,pisco,0,PE,1608481796,83,-13.70,-76.22,71.01,9.17
2,137,137,san juan,0,AR,1608481801,22,-31.54,-68.54,79.48,8.16
3,150,150,tahta,0,EG,1608481807,35,26.77,31.50,73.40,8.05
4,157,157,lam luk ka,0,TH,1608481763,64,13.93,100.75,75.00,5.82
5,264,264,viedma,0,AR,1608481830,31,-40.81,-63.00,75.00,1.99
6,322,322,cap malheureux,0,MU,1608481840,77,-19.98,57.61,79.00,8.01
7,491,491,grand gaube,0,MU,1608481868,77,-20.01,57.66,79.00,8.01
8,523,523,noumea,0,NC,1608481873,94,-22.28,166.46,78.01,1.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = focus_cities
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='index')
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed,Hotel Name
0,65,vaini,0,TO,1608481791,94,-21.20,-175.20,77.00,8.05,NaN
1,100,pisco,0,PE,1608481796,83,-13.70,-76.22,71.01,9.17,NaN
2,137,san juan,0,AR,1608481801,22,-31.54,-68.54,79.48,8.16,NaN
3,150,tahta,0,EG,1608481807,35,26.77,31.50,73.40,8.05,NaN
4,157,lam luk ka,0,TH,1608481763,64,13.93,100.75,75.00,5.82,NaN
5,264,viedma,0,AR,1608481830,31,-40.81,-63.00,75.00,1.99,NaN
6,322,cap malheureux,0,MU,1608481840,77,-19.98,57.61,79.00,8.01,NaN
7,491,grand gaube,0,MU,1608481868,77,-20.01,57.66,79.00,8.01,NaN
8,523,noumea,0,NC,1608481873,94,-22.28,166.46,78.01,1.12,NaN


In [11]:
hotels = []

x = 0

for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x])
    x+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)


hotel_df['Hotel Name'] = hotels
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed,Hotel Name
0,65,vaini,0,TO,1608481791,94,-21.20,-175.20,77.00,8.05,Keleti Beach Resort
1,100,pisco,0,PE,1608481796,83,-13.70,-76.22,71.01,9.17,Hotel San Isidro Oficial
2,137,san juan,0,AR,1608481801,22,-31.54,-68.54,79.48,8.16,Hotel Provincial
3,150,tahta,0,EG,1608481807,35,26.77,31.50,73.40,8.05,ماستر تك
4,157,lam luk ka,0,TH,1608481763,64,13.93,100.75,75.00,5.82,JUGA 1
5,264,viedma,0,AR,1608481830,31,-40.81,-63.00,75.00,1.99,Hotel Cristal
6,322,cap malheureux,0,MU,1608481840,77,-19.98,57.61,79.00,8.01,Pereybere Hotel & Spa
7,491,grand gaube,0,MU,1608481868,77,-20.01,57.66,79.00,8.01,Veranda Paul et Virginie Hotel & Spa
8,523,noumea,0,NC,1608481873,94,-22.28,166.46,78.01,1.12,Hôtel Le Lagon


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))